# IMPORT

In [15]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
import logging
import typing
import gc

In [2]:
# if you are running this notebook from 'home/dev/enefit/notebook'. 
os.chdir('..') # else adjust to point to the root of the project.

In [3]:
%%capture output

%load_ext kedro.ipython
%reload_kedro

if 'output' in locals() and 'error' in output.stderr:
    output.show()

# DATA TRANSFORMATION
Hereafter we transform the csv in parquet for perfomance improvement. visit https://parquet.apache.org/ for more information.

In [4]:
print(catalog.list())

['client', 'cl', 'electricity_prices', 'ep', 'forecast_weather', 'fw', 'gas_prices', 'gp', 'historical_weather', 'hw', 'train', 'tr', 'county_id_to_name_map', 'client_eg', 'electricity_prices_eg', 'forecast_weather_eg', 'gas_prices_eg', 'historical_weather_eg', 'revealed_targets', 'sample_submission', 'test', 'metrics', 'parameters']


## DATA DTYPES EXPLORATION

In [5]:
%%capture output

raw_datasets = {name: catalog.load(name) for name in catalog.list() if name != 'metrics'}

if 'output' in locals() and 'error' in output.stderr:
    output.show()

In [6]:
for key, value in raw_datasets.items():
    try:
        print(key, '\n', '===========', '\n', value.dtypes, '\n', '++++++', '\n\n')
    except:
        pass

client 
 product_type            int64
county                  int64
eic_count               int64
installed_capacity    float64
is_business             int64
date                   object
data_block_id           int64
dtype: object 
 ++++++ 


cl 
 product_type                        int64
county                              int64
eic_count                           int64
installed_capacity                float64
is_business                         int64
date                  datetime64[ns, UTC]
data_block_id                       int64
dtype: object 
 ++++++ 


electricity_prices 
 forecast_date     object
euros_per_mwh    float64
origin_date       object
data_block_id      int64
dtype: object 
 ++++++ 


ep 
 forecast_date    datetime64[ns, UTC]
euros_per_mwh                float64
origin_date      datetime64[ns, UTC]
data_block_id                  int64
dtype: object 
 ++++++ 


forecast_weather 
 latitude                             float64
longitude                            flo

## CONVERT FORMATS DATES AND SAVE IN PARQUET

In [7]:
# session.run(pipeline_name="data_formating")
pipelines["data_formating"].nodes # run kedro viz to see the details of this pipleline nodes


[
    Node(set_columns_as_date, ['client'], 'cl', 'convert_date_columns_in_CL'),
    Node(set_columns_as_date, ['electricity_prices'], 'ep', 'convert_date_columns_in_EP'),
    Node(set_columns_as_date, ['forecast_weather'], 'fw', 'convert_date_columns_in_FW'),
    Node(set_columns_as_date, 'gas_prices', 'gp', 'convert_date_columns_in_GP'),
    Node(set_columns_as_date, ['historical_weather'], 'hw', 'convert_date_columns_in_HW'),
    Node(set_columns_as_date, ['train'], 'tr', 'convert_date_columns_in_TR'),
    Node(convert_to_parquet, {'cl': 'cl', 'ep': 'ep', 'fw': 'fw', 'gp': 'gp', 'hw': 'hw', 'tr': 'tr'}, None, 'convert_to_parquet')
]

# DATA COMPREHENSION & EDA

## DATA COMPREHENSION 

In [8]:
%%capture output

pq_datasets = {name: catalog.load(name) for name in pipelines["data_formating"].all_outputs()}

if 'output' in locals() and 'error' in output.stderr:
    output.show()

<div class="alert alert-block alert-info">
<b>Tip:</b> Install VSCode Data Wranger extension for basic descriptives statistics about the data</div>

`data:cl:eic_count` -> The client (prosumer) can deliver energy to more than one other entity.
`data:cl:installed_capacity` -> solar panel capacity in kW.

`data:ep` fournit des infos sur le prix de l'énergie pour chaque heure.
`data:ep:euros_per_mwh` est rendu officielle a  `data:ep:origin_date`. cette information doit servir a anticiper le marché le lendemain a `data:ep:forecast_date` sur les marchés à terme.
<div class="alert alert-block alert-warning">
<b>?:</b> `data:ep:data_block_id` pour dire queles informations arrivent en batch ?
</div>


`data:gp`, A la différence de `data:ep`, fournit des infos du prix de du gaz naturel par jour (le plus haut et le plus bas).

## EDA

In [ ]:

tr, hw, ep, gp, fw, cl = pq_datasets.values()
del pq_datasets 
gc.collect()

In [32]:
epfig = go.Figure()
for block in ep['data_block_id'].unique():
    data = ep[ep['data_block_id'] == block]
    trace = go.Scatter(x=data['origin_date'],
                       y=data['euros_per_mwh'],
                       mode='lines',
                       name=f'Data Block {block}')
    epfig.add_trace(trace)

epfig.update_layout(title='Energy prices over Time',
                    xaxis_title='Origin Date',
                    yaxis_title='Euros per MWh')
epfig.show()

In [36]:
gpfig = go.Figure()
for block in gp['data_block_id'].unique():
    data = gp[gp['data_block_id'] == block]  
    traceh = go.Scatter(x=data['origin_date'],
                       y=data['highest_price_per_mwh'],
                       # mode='lines',
                       name=f'Data Block {block} high')
    tracel = go.Scatter(x=data['origin_date'],
                       y=data['lowest_price_per_mwh'],
                       mode='lines',
                       name=f'Data Block {block} low')
    gpfig.add_traces([traceh, tracel])
gpfig.update_layout(title='Gas Prices over Time',
                    xaxis_title='Origin Date',
                    yaxis_title='Euros per MWh')
gpfig.show()

In [26]:
gp['data_block_id'].unique()


array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 1